# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

## Unpickle the dictionary of past bump tests

In [ ]:
filename = '/scratch/cslage/m1m3_data/average_spline_dict_28apr23.pkl'
file = open(filename, 'rb')
averageSplineDict = pkl.load(file)
file.close()

## Now plot the residuals against the average

In [ ]:
async def plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id):
    [averagePrimarySpline, averageSecondarySpline] = averageSplineDict[id]
    thisBump = bumps[bumps['actuatorId']==id]
    timestamp = thisBump.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test with Residuals. Actuator ID {id}\n{timestamp}", fontsize=18)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primaryForces = forces[primaryForce].values
    primaryResiduals = primaryForces-averagePrimarySpline(times)
    primaryRmsError = np.sqrt(np.mean(primaryResiduals**2))
    plotStart -= t0
    plotEnd -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, averagePrimarySpline(times), label='Average')
    plt.plot(times, primaryForces, label='Data')
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(times, primaryResiduals)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 75.0, f"RMS = {primaryRmsError:.2f}", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 75.0, f"RMS = {primaryRmsError:.2f}", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-100,100)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Residuals (nt)")
    
    if secondaryName is not None:
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondaryForces = forces[secondaryForce].values
        secondaryResiduals = secondaryForces-averageSecondarySpline(times)
        secondaryRmsError = np.sqrt(np.mean(secondaryResiduals**2))
        plotStart -= t0
        plotEnd -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondaryName}")
        plt.plot(times, averageSecondarySpline(times), label='Average')
        plt.plot(times, secondaryForces, label='Data')
        if passFail[primaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.plot(times, secondaryResiduals)
        if passFail[primaryBump] == 6:
            plt.text(2.0, 75.0, f"RMS = {secondaryRmsError:.2f}", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 75.0, f"RMS = {secondaryRmsError:.2f}", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-100,100)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Residuals (nt)")
    else:
        secondaryRmsError = None
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return [primaryRmsError, secondaryRmsError]

# Now run all the actuators with the first test on the TMA

In [ ]:
# Times of bump test
start = Time("2023-04-28T18:10:00", scale='utc')
end = Time("2023-04-28T19:18:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/scratch/cslage/m1m3_data/Bump_Test{timestamp}.pdf")

for index in range(len(FATABLE)):
    try:
        id = FATABLE[index][M1M3FATable.FATABLE_ID]
        fig = plt.figure(figsize=(10,10))
        await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.close()
    except:
        continue
pdf.close()


In [ ]:
# Finding multiple tests form 20230504

start = Time("2023-05-03T00:00:00", scale='utc')
end = Time("2023-05-05T12:00:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

In [ ]:
for index in range(len(FATABLE)):
    try:
        id = FATABLE[index][M1M3FATable.FATABLE_ID]
        thisBump = bumps[bumps['actuatorId']==id]
        print(thisBump.index[0].isoformat().split(".")[0], "\t",  thisBump['timestamp'].values[0],"\t",id)
    except:
        continue

In [ ]:

pdf = PdfPages(f"/scratch/cslage/m1m3_data/Bump_Test_20230504.pdf")
times = [[Time("2023-05-04T14:28:00", scale='utc'), Time("2023-05-04T14:35:00", scale='utc')], \
        [Time("2023-05-05T00:00:00", scale='utc'), Time("2023-05-05T00:45:00", scale='utc')], \
         [Time("2023-05-05T03:00:00", scale='utc'), Time("2023-05-05T04:00:00", scale='utc')]]

for [start, end] in times:
    bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)
         
    for index in range(len(FATABLE)):
        try:
            id = FATABLE[index][M1M3FATable.FATABLE_ID]
            fig = plt.figure(figsize=(10,10))
            await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
            pdf.savefig(fig)  # saves the current figure into a pdf page
            plt.close()
        except:
            continue
pdf.close()


In [ ]:
# Finding multiple tests form 20230506

start = Time("2023-05-05T12:00:00", scale='utc')
end = Time("2023-05-06T20:00:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

In [ ]:
thisBump = bumps[bumps['actuatorId']==101]

In [ ]:
thisBump

In [ ]:
%matplotlib inline
names = ['Last Level 3 Test', 'First TMA Test', '06May #1', '06May #2', '06May #3', '06May #4']
times = [[Time("2023-04-19T12:50:00", scale='utc'), Time("2023-04-19T14:50:00", scale='utc')], \
        [Time("2023-04-28T18:10:00", scale='utc'), Time("2023-04-28T19:30:00", scale='utc')], \
        [Time("2023-05-06T00:20:00", scale='utc'), Time("2023-05-06T01:34:00", scale='utc')], \
        [Time("2023-05-06T01:35:00", scale='utc'), Time("2023-05-06T03:47:00", scale='utc')], \
         [Time("2023-05-06T03:47:00", scale='utc'), Time("2023-05-06T06:00:00", scale='utc')], \
        [Time("2023-05-06T18:50:00", scale='utc'), Time("2023-05-06T22:00:00", scale='utc')]]

rmsErrors = {}
for [start, end] in times:
    ids = []
    primaries = []
    secondaries = []
    bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)
    timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
    pdf = PdfPages(f"/scratch/cslage/m1m3_data/Bump_Test_{timestamp}.pdf")
    for index in range(len(FATABLE)):
        try:
            id = FATABLE[index][M1M3FATable.FATABLE_ID]
            fig = plt.figure(figsize=(10,10))
            [primaryRmsError, secondaryRmsError] = await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
            ids.append(id)
            primaries.append(primaryRmsError)
            secondaries.append(secondaryRmsError)
            pdf.savefig(fig)  # saves the current figure into a pdf page
            plt.close()
        except:
            continue
    pdf.close()
    rmsErrors[timestamp] = [ids, primaries, secondaries]


In [ ]:
len(rmsErrors)

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)


for i, key in enumerate(rmsErrors.keys()):
    ii = i%3
    jj = int(i/3)
    [ids, primaries, secondaries] = rmsErrors[key]
    axs[ii][jj].set_title(names[i])
    axs[ii][jj].scatter(ids, primaries, marker='x', color='red', label='Primary')
    axs[ii][jj].scatter(ids, secondaries, marker='+', color='blue', label='Secondary')
    for n, id in enumerate(ids):
        if primaries[n] is not None:
            if primaries[n] > 25.0:
                axs[ii][jj].text(id+2, primaries[n], f"{id}", color='red', fontsize=8)
        if secondaries[n] is not None:
            if secondaries[n] > 25.0:
                axs[ii][jj].text(id+2, secondaries[n], f"{id}", color='blue', fontsize=8)
    axs[ii][jj].set_ylim(0,120) 
    axs[ii][jj].set_xlim(100, 450)
    axs[ii][jj].set_xlabel('Actuator ID')
    axs[ii][jj].set_ylabel('RMS deviation from average (nt)')
axs[0][0].legend(loc='upper left')
plt.savefig(f"/scratch/cslage/m1m3_data/Bump_Test_Summary_06May23.png")